In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [13]:
capacity = pd.read_csv('https://raw.githubusercontent.com/johnson80245/python/refs/heads/main/csvs/theoretical%20capacity%20data%203%20months.csv')
all_state_aht = pd.read_csv('https://raw.githubusercontent.com/johnson80245/python/refs/heads/main/csvs/AHT_by_state.csv')
def to_clean(val):
    return val.strip().lower().replace(" ","_")

capacity = capacity.rename(columns = to_clean)
all_state_aht = all_state_aht.rename(columns = to_clean)

In [14]:
capacity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year_month            91 non-null     object 
 1   userid                91 non-null     int64  
 2   fte                   91 non-null     float64
 3   position              91 non-null     object 
 4   team_lead_id          91 non-null     int64  
 5   total_filings         91 non-null     int64  
 6   flag_count            91 non-null     int64  
 7   srt_reassignments     91 non-null     int64  
 8   filing_admin_changes  91 non-null     int64  
 9   new_policy_count      91 non-null     int64  
 10  renewal_count         91 non-null     int64  
 11  endorsement__count    91 non-null     int64  
 12  srt_total             91 non-null     int64  
 13  premium_srt           91 non-null     int64  
 14  zero_srt              91 non-null     int64  
 15  no_charge_srt         91 

In [15]:
#Cast as categorical data
capacity[['userid','position','team_lead_id']] = (capacity[['userid','position','team_lead_id']].astype('category'))
#Cast to float
capacity['adj_total_hours'] = pd.to_numeric(capacity.adj_total_hours, errors = 'coerce')
#Cast to date
capacity['year_month'] = pd.to_datetime(capacity['year_month'])
#Cast to int64
int_columns = ['new_policy_count', 'renewal_count', 'endorsement__count','fte','total_filings','flag_count','srt_reassignments',
               'filing_admin_changes','srt_total','premium_srt','zero_srt','no_charge_srt','ad_hoc_srts']
for column in int_columns:
    capacity[column] = capacity[column].astype('int64')




In [16]:
#Create AHT based on reported hours
capacity['filing_aht']  = (capacity.filing_hours_self / capacity.total_filings) * 60
capacity['srt_aht']  = (capacity.srt_hours_self / capacity.srt_total) * 60
capacity['flag_aht'] = (capacity['filing_aht'] * 1.25)

#Create Utilization
capacity['filing_utilization'] = (capacity.filing_hours_self/capacity.total_hours_self) 
capacity['srt_utilization'] = (capacity.srt_hours_self/capacity.total_hours_self) 
capacity['total_utilization'] = (capacity.filing_hours_self + capacity.srt_hours_self) / capacity.total_hours_self

In [17]:
all_state_aht.dtypes

state                      object
submission_type            object
transaction_type_group     object
transaction_count          object
transaction_share         float64
weighted_aht              float64
dtype: object

In [18]:
# Correct way
all_state_aht['transaction_count'] = pd.to_numeric(
    all_state_aht['transaction_count'].astype(str).str.replace(',', ''),
    errors='coerce'
).astype('float64')


In [ ]:
#Create state weighted aht

def compute_weighted_aht(group):
    return (
        (group['transaction_count'] * group['weighted_aht']).sum()
        / group['transaction_count'].sum()
    )

state_trans_aht = (
    all_state_aht
    .groupby(['state', 'transaction_type_group'], group_keys=False)
    .apply(compute_weighted_aht, include_groups=False)
    .reset_index(name='weighted_aht')
)

state_aht = (
    all_state_aht
    .groupby('state', group_keys = False)
    .apply(compute_weighted_aht, include_groups=False)
    .reset_index(name = 'weighted_aht')
)

filing_perc_all = all_state_aht.groupby(['state', 'submission_type'])['transaction_share'].sum().reset_index()
filing_perc_st_group = all_state_aht.groupby(['state','transaction_type_group'])['transaction_share'].sum().reset_index()
filing_perc_st_submission = all_state_aht.groupby(['state', 'submission_type'])['transaction_share'].sum().reset_index()
filing_perc_submission = all_state_aht.groupby(['submission_type'])['transaction_share'].sum().reset_index()
filing_perc_trans_group = all_state_aht.groupby(['transaction_type_group'])['transaction_share'].sum().reset_index()


,submission_type,transaction_share
0,Batch,0.657943
1,Manual,0.342057


In [ ]:
filing_perc_st_submission.head()

,state,submission_type,transaction_share
0,AK,Manual,0.001573
1,AL,Batch,0.004865
2,AL,Manual,0.000485
3,AR,Manual,0.005324
4,AZ,Batch,0.019694


In [ ]:
pd.set_option('display.max_columns', 60)
capacity.sample(5,random_state = 13)

capacity

,year_month,userid,fte,position,team_lead_id,total_filings,flag_count,srt_reassignments,filing_admin_changes,new_policy_count,renewal_count,endorsement__count,srt_total,premium_srt,zero_srt,no_charge_srt,ad_hoc_srts,total_hours_self,pto_hours_self,adj_total_hours,filing_hours_self,srt_hours_self,filing_aht,srt_aht,flag_aht,filing_utilization,srt_utilization,total_utilization
33,2025-06-01,653,1,Tax Associate,223,303,47,0,130,245,18,40,0,0,0,0,0,108.05,14.5,93.55,75.0,0.00,14.851485,NaN,18.564356,0.694123,0.000000,0.694123
23,2025-04-01,807,1,Tax Associate,223,1029,183,298,310,306,112,611,12,10,2,0,3,176.00,20.0,156.00,141.0,3.50,8.221574,17.500000,10.276968,0.801136,0.019886,0.821023
37,2025-06-01,397,1,Tax Associate,467,976,87,61,124,565,164,247,41,18,23,0,8,96.00,16.0,80.00,51.0,8.25,3.135246,12.073171,3.919057,0.531250,0.085938,0.617188
79,2025-03-01,172,1,Tax Associate,772,1355,72,17,719,1043,75,237,88,61,27,0,0,186.75,12.0,174.75,44.5,91.00,1.970480,62.045455,2.463100,0.238286,0.487282,0.725569
14,2025-03-01,128,1,Tax Analyst,467,436,155,24,127,174,90,172,81,6,75,0,0,165.50,2.0,163.50,88.5,47.25,12.178899,35.000000,15.223624,0.534743,0.285498,0.820242
